In [ ]:
### imports

# external modules
import sys
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import importlib

# local modules
sys.path.append(os.path.abspath('../../ML4DQMDC-PixelAE/omsapi'))
import get_oms_data
importlib.reload(get_oms_data)
from get_oms_data import get_oms_api, get_oms_data
from get_oms_data import get_oms_response_attributes, get_oms_response_attribute
sys.path.append(os.path.abspath('../../ML4DQMDC-PixelAE'))
import utils.plot_utils
importlib.reload(utils.plot_utils)
from utils.plot_utils import plot_hists

In [ ]:
# get the omsapi instance

omsapi = get_oms_api()

In [ ]:
# define run ranges

# references:
# - https://twiki.cern.ch/twiki/bin/view/CMS/PdmVRun3Analysis#2023_Era_definition
# - DAS (query runs for each ZeroBias DQMIO dataset)
eradict = {
  'Run2023C-v1': (367080, 367515), # (DAS: start from 367094)
  'Run2023C-v2': (367516, 367620), # (DAS: stop at 367758, but this is incorrect in DAS client, actual data does not overlap with v3)
  'Run2023C-v3': (367621, 367763), # (DAS: start at 367622, stop at 367758)
  'Run2023C-v4': (367765, 369802), # (DAS: start at 367770)
  'Run2023D-v1': (369803, 370602), # (DAS: start from 369827, stop at 370580)
  'Run2023D-v2': (370603, 372415), # (DAS: start from 370616, stop at 371227)
  'Run2023E': (372417, 373075), # (DAS: start from 372474, stop at 373072)
  'Run2023F': (373076, 373784) # (DAS: start from 373078, stop is from DAS as Twiki is not yet filled)
}

In [ ]:
# find out which attributes are available per lumisection

runnb = 367515 # dummy run, should not matter
ls_info = get_oms_data( omsapi, 'lumisections', runnb )
print( get_oms_response_attributes(ls_info) )

In [ ]:
# define attributes to retrieve

attributes = [
    'delivered_lumi_per_lumisection',
    'recorded_lumi_per_lumisection',
    'pileup',
    'physics_flag',
    'fill_number',
    'run_number',
    'lumisection_number',
]

In [ ]:
# define an auxiliary function to get the data from OMS

def get_data_iterative(omsapi, endpoint, runrange, attributes, **kwargs):
    # note: the maximum allowed limit in OMS calls seems to be 100k lumisections,
    #       but some eras have more, so need to split run range in parts.
    limit_entries = 100000 # hard limit in OMS API
    ls_info_probe = get_oms_data( omsapi, 'lumisections', runrange, attributes=['lumisection_number'], limit_entries=limit_entries )
    if len(get_oms_response_attribute(ls_info_probe,'lumisection_number')) < limit_entries:
        ls_info_raw = get_oms_data( omsapi, endpoint, runrange, attributes=attributes, limit_entries=limit_entries, **kwargs )
        ls_info = {attribute: get_oms_response_attribute(ls_info_raw, attribute) for attribute in attributes}
        return ls_info
    else:
        midrange = int((runrange[0]+runrange[1])/2)
        firstrange = (runrange[0], midrange)
        ls_info_1 = get_data_iterative(omsapi, endpoint, firstrange, attributes)
        secondrange = (midrange+1, runrange[1])
        ls_info_2 = get_data_iterative(omsapi, endpoint, secondrange, attributes)
        ls_info = {attribute: ls_info_1[attribute]+ls_info_2[attribute] for attribute in attributes}
        return ls_info

In [ ]:
# get the data from OMS

info = {}
for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    ls_info = get_data_iterative(omsapi, 'lumisections', runrange, attributes)
    print('Found {} lumisections'.format(len(ls_info[list(ls_info.keys())[0]])))
    info[era] = ls_info

In [ ]:
# make plots
# note: the unit of luminosity per lumisection is inverse pb, as retrieved from looking at OMS tables.

for era in eradict.keys():
    fig, ax = plt.subplots(figsize=(18,6))
    thisinfo = info[era]
    plot_hists([thisinfo['delivered_lumi_per_lumisection'], thisinfo['recorded_lumi_per_lumisection']],
               fig=fig, ax=ax,
               colorlist=['b','g'],
               labellist=['Delivered lumi', 'Recorded lumi'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Luminosity (/pb)', yaxtitlesize=15,
               title='Luminosity for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=thisinfo['physics_flag'], bkgcmap='spring')
    fig, ax = plt.subplots(figsize=(18,6))
    plot_hists([thisinfo['pileup']],
               fig=fig, ax=ax,
               colorlist=['b','g'],
               labellist=['Pileup'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Pileup', yaxtitlesize=15,
               title='Pileup for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=thisinfo['physics_flag'], bkgcmap='spring')

In [ ]:
# save to json files
skip = True

if not skip:

    for era in eradict.keys():
        thisinfo = info[era]
        outputfile = 'omsdata/omsdata_{}.json'.format(era)
        with open(outputfile, 'w') as f:
            json.dump(thisinfo, f)

In [ ]:
# get available HLT paths and attributes

#runnb = 367515 # dummy run, should not matter
runnb = 367589
ls_info = get_oms_data( omsapi, 'hltpathinfo', runnb=runnb, attributes=['path_name'] )
hltpaths = get_oms_response_attribute(ls_info,'path_name')
hltpaths = [p for p in hltpaths if p.startswith('HLT_ZeroBias')]
print('Available ZeroBias triggers:')
for p in hltpaths: print('  {}'.format(p))

path_filter = {'attribute_name':'path_name','value':'HLT_ZeroBias_v7','operator':'EQ'}
per_lumi_arg = {'group[granularity]':'lumisection'}
ls_info_raw = get_oms_data( omsapi, 'hltpathrates', 367515, extraargs=per_lumi_arg, extrafilters=[path_filter])
print('Available attributes')
print(get_oms_response_attributes(ls_info_raw))

In [ ]:
# get the HLT rate from OMS
# note: only works for filtering per run, not run ranges.

dosave = True

attributes = [
    'fill_number',
    'run_number',
    'first_lumisection_number',
    'rate'
]

info = {}
for era, runrange in eradict.items():
    if era != 'Run2023C-v2': continue
    print('Retrieving data for era {}'.format(era))
    # get a list of runs
    runs = get_oms_data( omsapi, 'runs', runnb=runrange, attributes=['run_number'] )
    runs = get_oms_response_attribute(runs, 'run_number')
    print('Looping over {} runs'.format(len(runs)))
    # loop over runs
    for run in runs:
        # get the zerobias name for this run
        hltinfo = get_oms_data( omsapi, 'hltpathinfo', runnb=run, attributes=['path_name'], limit_entries=10000 )
        hltpaths = get_oms_response_attribute(hltinfo,'path_name')
        zerobiaspaths = [p for p in hltpaths if p.startswith('HLT_ZeroBias_v')]
        if len(zerobiaspaths)==0:
            print('WARNING: no ZeroBias HLT path found for run {}; skipping this run.'.format(run))
            continue
        if len(zerobiaspaths)>1:
            print('WARNING: more than one ZeroBias HLT path found for run {}; picking the first.'.format(run))
            continue
        zerobiaspath = zerobiaspaths[0]
        # get the trigger rate for this run and this hlt path
        path_filter = {'attribute_name':'path_name','value':zerobiaspath,'operator':'EQ'}
        per_lumi_arg = {'group[granularity]':'lumisection'}
        ls_info_raw = get_oms_data( omsapi, 'hltpathrates', runnb=run, attributes=attributes,
                                extraargs=per_lumi_arg, extrafilters=[path_filter], limit_entries=10000 )
        ls_info = {attribute: get_oms_response_attribute(ls_info_raw, attribute) for attribute in attributes}
        print('Found {} lumisections'.format(len(ls_info[list(ls_info.keys())[0]])))
        info[run] = ls_info
        if dosave:
            outputfile = 'omsdata/hltrate_{}.json'.format(run)
            with open(outputfile, 'w') as f:
                json.dump(info[run], f)

In [ ]:
# make plots
# note: work on the basis of previously stored json files,
# as the cell above is too instable and slow to get all data in one go.

alljsonfiles = sorted([f for f in os.listdir('omsdata') if f.startswith('hltrate_')])
allruns = [int(f.replace('hltrate_','').replace('.json','')) for f in alljsonfiles]

for era,runrange in eradict.items():
    if era != 'Run2023C-v2': continue
    hltrates = []
    runboundaries = {}
    globalidx = 0
    # loop over correct runs for this era
    eraruns = [run for run in allruns if (run >= runrange[0] and run <= runrange[1])]
    for run in eraruns:
        jsonfile = 'omsdata/hltrate_{}.json'.format(run)
        # load the data
        with open(jsonfile, 'r') as f:
            data = json.load(f)['rate']
            hltrates.append(data)
            runboundaries[run] = globalidx
            globalidx += len(data)
    hltrate = np.concatenate(hltrates)
    fig, ax = plt.subplots(figsize=(18,6))
    plot_hists([hltrate],
               fig=fig, ax=ax,
               colorlist=['b'],
               labellist=['HLT ZeroBias rate'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='HLT rate (/s)', yaxtitlesize=15,
               title='HLT ZeroBias rate for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None)
    # add lines and run numbers
    txtheight = ax.get_ylim()[1]*0.98
    txtlowershift = (ax.get_ylim()[1]-ax.get_ylim()[0])*0.05
    currentlowershift = 0
    runidx = list(runboundaries.items())
    for i in range(len(runidx)):
        run = runidx[i][0]
        idx = runidx[i][1]
        ax.axvline(idx, color='k', linestyle='--')
        if i>0:
            if idx > runidx[i-1][1]+500: currentlowershift = 0
            else: currentlowershift += 1
            if currentlowershift>3: currentlowershift = 0
        txt = ax.text(idx+25, txtheight-currentlowershift*txtlowershift, run, va='top', fontsize=12)
        txt.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='white'))